In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv("./data/the-movies-dataset/ratings_small.csv")# 100k small data set

In [3]:
dataset.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
len(dataset.userId.unique()), len(dataset.movieId.unique())

(671, 9066)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
### trainform data type int to object ( userId, movieId)
dataset.userId = dataset.userId.astype('category')
dataset.movieId = dataset.movieId.astype('category')

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null category
movieId      100004 non-null category
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: category(2), float64(1), int64(1)
memory usage: 2.3 MB


## Model creation

In [8]:
import tensorflow as tf

In [9]:
n_latent_factor_user = 8
n_latent_factor_moive = 10
n_latent_factor_mf = 3
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())

In [10]:
dataset.movieId.unique()

[31, 1029, 1061, 1129, 1172, ..., 64997, 72380, 129, 4736, 6425]
Length: 9066
Categories (9066, int64): [31, 1029, 1061, 1129, ..., 72380, 129, 4736, 6425]

In [11]:
# movie_feature_column  = tf.feature_column.categorical_column_with_vocabulary_list(key='terms', vocabulary_list=dataset.movieId.unique())
from tensorflow import keras

In [12]:
movie_input = keras.layers.Input(shape = [1], name = 'Movie')
movie_embedding_mlp = keras.layers.Embedding(n_movies +1 , n_latent_factor_moive, name = 'Movie_embedding_MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name = 'FlattenMovies_MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies+1 , n_latent_factor_mf, name = 'Movie_embedding_MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies_MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)

user_input = keras.layers.Input(shape=[1],name='User')
user_embedding_mlp = keras.layers.Embedding(n_users +1 , n_latent_factor_user, name = 'Users_embedding_MLP')(user_input)
user_vec_mlp = keras.layers.Flatten(name = 'FlattenUser_MLP')(user_embedding_mlp)
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_embedding_mf = keras.layers.Embedding(n_users +1 , n_latent_factor_mf, name = 'Users_embedding_MF')(user_input)
user_vec_mf = keras.layers.Flatten(name = 'FlattenUser_MF')(user_embedding_mf)
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)


concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp])
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense_1 = keras.layers.Dense(200, name ='FullyConnected_1')(concat_dropout)
dense_batch_1 = keras.layers.BatchNormalization(name='Batch')(dense_1)
dropout_1 = keras.layers.Dropout(0.2, name ='Dropout_1')(dense_batch_1)
dense_2 = keras.layers.Dense(100, name = 'FullyConnected_2')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name = 'Batch_2')(dense_2)

dropout_2 = keras.layers.Dropout(0.2, name='Dropout_2')(dense_batch_2)
dense_3 = keras.layers.Dense(50, name = 'FullyConnected_3')(dropout_2)
dense_4 = keras.layers.Dense(20, name = 'FullyConnected_4', activation='relu')(dense_3)

In [13]:
(movie_vec_mlp.shape , user_vec_mlp.shape)

(TensorShape([Dimension(None), Dimension(10)]),
 TensorShape([Dimension(None), Dimension(8)]))

In [14]:
concat.shape

TensorShape([Dimension(None), Dimension(18)])

In [15]:
(user_vec_mf.shape, movie_vec_mf.shape)

(TensorShape([Dimension(None), Dimension(3)]),
 TensorShape([Dimension(None), Dimension(3)]))

In [16]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [17]:
user_vec_mf

<tf.Tensor 'dropout_3/cond/Merge:0' shape=(?, 3) dtype=float32>

In [18]:
# movie_vec_mf = keras.backend.transpose(movie_vec_mf) # movie_vec_mf shape=(?, 3) to (3,?)

In [19]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [20]:
pred_mf = keras.layers.multiply([movie_vec_mf, user_vec_mf], name = 'Elemet_wise_product')
pred_mlp = keras.layers.Dense(1, activation='relu', name = 'Activation_pred')(dense_4)

In [21]:
(pred_mf, pred_mlp)

(<tf.Tensor 'Elemet_wise_product/mul:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'Activation_pred/Relu:0' shape=(?, 1) dtype=float32>)

In [22]:
combine_mlp_mf = keras.layers.concatenate([pred_mf,pred_mlp],name = 'Concat_MF_MLP')
result_combine = keras.layers.Dense(100, name='Combine_MF_MLP')(combine_mlp_mf)

result = keras.layers.Dense(1, name='Prediction')(result_combine)

model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr = 0.01)
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [23]:
from IPython.display import SVG

from tensorflow.keras.utils import plot_model

In [28]:
SVG(plot_model(model, show_shapes=False,to_file='test_model', show_layer_names=True, rankdir='TB')) 
# SVG(plot_model(model, show_shapes=False,to_file='test_model', show_layer_names=True, rankdir='TB').create(prog='dot',format='svg'))

<IPython.core.display.SVG object>

In [29]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie_embedding_MLP (Embedding) (None, 1, 10)        90670       Movie[0][0]                      
__________________________________________________________________________________________________
Users_embedding_MLP (Embedding) (None, 1, 8)         5376        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov